In [ ]:
import csv
import json
import time
import heapq
from datetime import datetime, timedelta
from math import sqrt
import math
import random

In [ ]:
def read_csv_to_matrix(file_path):
    data_matrix = []

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        for row in csv_reader:
            data_matrix.append(row)

    return data_matrix

csv_file_path = 'drivers.csv'
drivers_matrix = read_csv_to_matrix('drivers.csv')
passengers_matrix = read_csv_to_matrix('passengers.csv')
edges_matrix = read_csv_to_matrix('edges.csv')

In [ ]:
print("Drivers")
for i, row in enumerate(drivers_matrix[:5]):
    print(row)
print("Passengers")
for i, row in enumerate(passengers_matrix[:5]):
    print(row)
print("Edges")
for i, row in enumerate(edges_matrix[:5]):
    print(row)

In [ ]:
def read_json_to_dict(file_path):
    with open(file_path, 'r') as json_file:
        data_dict = json.load(json_file)

    return data_dict

json_file_path = 'node_data.json'
node_data_dict = read_json_to_dict(json_file_path)

count = 0
for node_id, coordinates in node_data_dict.items():
    print(f'Node ID: {node_id}, Coordinates: {coordinates}')
    count += 1
    if count == 5:
        break

# T1 
- Create quques to keep track of available drivers and passengers 
- When a driver is free, assign longest waiting passenger to them (the one with the earliest date/time)
- Disregard distances
- When matched, remove driver and passenger from the queue
- update to recycle drivers (find a way to estimate how long each ride takes-- use the graph?)

In [ ]:
def match_passenger_to_driver(drivers, passengers):
    start_time = time.time()
    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)
    heapq.heapify(passenger_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        current_passenger_time, current_passenger = heapq.heappop(passenger_heap)
        # remove conditional, assign first passenger to first driver
        print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")
        drive_duration = timedelta(minutes=20)
        # requeue driver 
        heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
 #04/25/2014 07:00:00
match_passenger_to_driver(drivers_matrix[1:], passengers_matrix[1:])

How do we simulate that the ride is over for a driver passenger par so we can adsign that driver to a new passenger? This would be the case of T1, 2 and 3. 
- We were thinking we could assign an arbitrary time for each ride to take place, then add the driver back into the queue 
- Or we can use a heuristic to estimate time of the ride depending on the distance between the coordinates (the straight line distance)
- Or should we pre process the graph for all of the tasks? Have an accurate calculation of the time the ride takes to finish? 
- Do we even have to consider the ride finishing logic? 



In [ ]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Calculate the Euclidean distance between two sets of coordinates
    dx = lon2 - lon1
    dy = lat2 - lat1
    distance = sqrt(dx**2 + dy**2)
    return distance

def match_passenger_to_driver_t2(drivers, passengers):
    start_time = time.time()
    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)
    # heapq.heapify(passenger_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        dist = float('inf')
        current_passenger = 0
        #count = -1
        #current_passenger_ind = 0
        for passenger in passenger_heap:
            #count = count + 1
            if passenger[0] < current_driver_time:
                if euclidean_distance(float(current_driver[1]),float(current_driver[2]),float(passenger[1][1]),float(passenger[1][2])) < dist:
                    dist = euclidean_distance(float(current_driver[1]),float(current_driver[2]),float(passenger[1][1]),float(passenger[1][2]))
                    current_passenger = passenger
                    #current_passenger_ind = count
        if current_passenger != 0:
            passenger_heap.remove(current_passenger)
        # remove conditional, assign first passenger to first driver
            print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger[1]}")
        drive_duration = timedelta(minutes=20)
        # requeue driver 
        heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")
 #04/25/2014 07:00:00
match_passenger_to_driver_t2(drivers_matrix[1:], passengers_matrix[1:])

## PreProcessing the Graph

creating a node dictionary, an edges dictionary, and an adjacency dictionary

In [ ]:
def read_graph_nodes_and_edges():
    with open('node_data.json') as nodes_file:
        nodes = json.load(nodes_file)

    edges = {}
    with open('edges.csv', 'r') as edges_csv:
        edges_reader = csv.DictReader(edges_csv)
        for row in edges_reader:
            start_id = int(row['start_id'])
            end_id = int(row['end_id'])
            length = float(row['length'])

            # Additional attributes for each hour of the day
            weekday_hours = [float(row[f'weekday_{i}']) for i in range(24)]
            weekend_hours = [float(row[f'weekend_{i}']) for i in range(24)]

            edge = (start_id, end_id)

            edges[edge] = {
                'length': length,
                'weekday_hours': weekday_hours,
                'weekend_hours': weekend_hours
            }

    return nodes, edges
nodes, edges = read_graph_nodes_and_edges()

# Display the result
print("Graph Nodes:")

for node_id, node_data in list(nodes.items())[:5]:
    print(f'Node ID: {node_id}, Node Data: {node_data}')

print("\nGraph Edges:")
for edge, attributes in list(edges.items())[:5]:
    print(f'Edge: {edge}, Attributes: {attributes}')

In [ ]:
# Create clusters graph 
def cluster_nodes(nodes, decimal_places=3):
    clusters = {}
    for node_id, coordinates in nodes.items():
        truncated_lat = round(coordinates['lat'], 2)
        truncated_lon = round(coordinates['lon'], 3)

        # Use a tuple (truncated_lat, truncated_lon) as the cluster identifier
        cluster_key = (truncated_lat, truncated_lon)

        if cluster_key not in clusters:
            clusters[cluster_key] = []

        clusters[cluster_key].append(node_id)

    return clusters

clusters = cluster_nodes(nodes)

def update_edges_between_clusters(clusters, nodes):
    adjacency_dict = {}
    for edge, attributes in edges.items():
        source_node, target_node = edge
        source_node = nodes[str(source_node)]
        target_node = nodes[str(target_node)]

        cluster_key1 = (round(source_node['lat'], 2),round(source_node['lon'], 3))
        cluster_key2 = (round(target_node['lat'], 2),round(target_node['lon'], 3))

        #print(cluster_key1,cluster_key2)
        # check if source and target are in different clusters aka they have different cluster keys 
        # if they are different, add it to the graph as an edge
        if cluster_key1 != cluster_key2:
            length = attributes["length"]
            speeds = attributes['weekday_hours']
            average_speed = sum(speeds)/len(speeds)
            estimated_time = length / average_speed
            # add this edge to the adjacency
            # Add source to target with weight
        if cluster_key1 in adjacency_dict:
            if cluster_key2 in adjacency_dict[cluster_key1]:
                adjacency_dict[cluster_key1][cluster_key2].append(estimated_time)
            else:
                adjacency_dict[cluster_key1][cluster_key2] = [estimated_time]
        else:
            adjacency_dict[cluster_key1] = {cluster_key2: [estimated_time]}

        # Ensure target node is in the dictionary, even if it has no outgoing edges
        if cluster_key2 not in adjacency_dict:
            adjacency_dict[cluster_key2] = {}
    return adjacency_dict

adjacency = update_edges_between_clusters(clusters, nodes)


for key1, nested_dict in adjacency.items():
        # Iterate over each key-value pair in the second level of the dictionary
        for key2, value in nested_dict.items():            
            # Calculate the average of the nested lists
            avg_list = sum(value) / len(value) 
            
            # Update the nested dictionary with the average list
            adjacency[key1][key2] = avg_list

print(len(adjacency))
for key, value in list(adjacency.items())[:5]:
    print(key, value)

In [ ]:
def dijkstras(graph, start, target):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node == target:
            break

        if current_distance <= distances[current_node]:
            for neighbor, weight in graph[float(current_node)].items():
                distance = current_distance + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))

    return distances[float(target)]

In [ ]:
for node in list(nodes.items())[:5]:
    print(node)

In [ ]:
coordinates_dict = {}

for node_id, coordinates in nodes.items():
    lon = coordinates['lon']
    lat = coordinates['lat']
    key = (lon, lat)
    coordinates_dict[key] = node_id

for node in list(coordinates_dict.items())[:5]:
    print(node)

In [ ]:
def find_closest_coordinates(target_coordinates, known_coordinates):
    min_distance = float('inf')
    closest_coordinates = None

    for key, value in known_coordinates.items():
        distance = euclidean_distance(target_coordinates[0], target_coordinates[1], value['lat'], value['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_coordinates = key

    return closest_coordinates


In [ ]:

def match_passenger_to_driver_t3(drivers, passengers, adjacency_dict, nodes):
    start_time = time.time()

    # Convert dates to datetime objects and initialize priority queues
    driver_heap = [(datetime.strptime(driver[0], '%m/%d/%Y %H:%M:%S'), driver) for driver in drivers]
    passenger_heap = [(datetime.strptime(passenger[0], '%m/%d/%Y %H:%M:%S'), passenger) for passenger in passengers]

    # Convert lists to heaps
    heapq.heapify(driver_heap)

    # Match drivers to passengers based on earliest date/time
    while driver_heap and passenger_heap:
        current_driver_time, current_driver = heapq.heappop(driver_heap)
        shortest_path_time = float('inf')
        current_passenger = None
        current_passenger_time = None

        for passenger_time, passenger in passenger_heap:
            if passenger_time < current_driver_time:
                #print((float(current_driver[1]), float(current_driver[2])))
                # Get the node IDs from the latitude and longitude using the Nodes dictionary
                start_node = find_closest_coordinates((float(current_driver[1]), float(current_driver[2])),nodes)
                end_node = find_closest_coordinates((float(passenger[1]), float(passenger[2])),nodes)

                path_time = dijkstras(adjacency_dict, start_node, end_node)

                if path_time < shortest_path_time:
                    shortest_path_time = path_time
                    current_passenger = passenger
                    current_passenger_time = passenger_time

        if current_passenger:
            print(current_passenger)
            passenger_heap.remove((current_passenger_time,current_passenger))
            current_driver[1] = current_passenger[3]
            current_driver[2] = current_passenger[4]
            print(f"Driver assigned to Passenger: {current_driver} -> {current_passenger}")

            # Introduce a 95% chance for the driver to be pushed back
        if random.random() < 0.2:
            drive_duration = timedelta(hours=shortest_path_time)
            heapq.heappush(driver_heap, (current_driver_time + drive_duration, current_driver))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Matching process complete. Elapsed time: {elapsed_time} seconds")

# Example usage
# Assuming you have an adjacency_dict, drivers_matrix, passengers_matrix, and Nodes dictionary
match_passenger_to_driver_t3(drivers_matrix[1:], passengers_matrix[1:], adjacency, nodes)


In [ ]:
print(adjacency_dict[1942055181])